In [1]:
import numpy as np
import pandas as pd 
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
class UkrainianStemmer():
    def __init__(self, word):
        self.word = word
        self.vowel = r'аеиоуюяіїє'  # http://uk.wikipedia.org/wiki/Голосний_звук
        self.perfectiveground = r'(ив|ивши|ившись|ыв|ывши|ывшись((?<=[ая])(в|вши|вшись)))$'
        # http://uk.wikipedia.org/wiki/Рефлексивне_дієслово
        self.reflexive = r'(с[яьи])$'
        # http://uk.wikipedia.org/wiki/Прикметник + http://wapedia.mobi/uk/Прикметник
        self.adjective = r'(ими|ій|ий|а|е|ова|ове|ів|є|їй|єє|еє|я|ім|ем|им|ім|их|іх|ою|йми|іми|у|ю|ого|ому|ої)$'
        # http://uk.wikipedia.org/wiki/Дієприкметник
        self.participle = r'(ий|ого|ому|им|ім|а|ій|у|ою|ій|і|их|йми|их)$'
        # http://uk.wikipedia.org/wiki/Дієслово
        self.verb = r'(сь|ся|ив|ать|ять|у|ю|ав|али|учи|ячи|вши|ши|е|ме|ати|яти|є)$'
        # http://uk.wikipedia.org/wiki/Іменник
        self.noun = r'(а|ев|ов|е|ями|ами|еи|и|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я|і|ові|ї|ею|єю|ою|є|еві|ем|єм|ів|їв|ю)$'
        self.rvre = r'[аеиоуюяіїє]'
        self.derivational = r'[^аеиоуюяіїє][аеиоуюяіїє]+[^аеиоуюяіїє]+[аеиоуюяіїє].*(?<=о)сть?$'
        self.RV = ''

    def ukstemmer_search_preprocess(self, word):
        word = word.lower()
        word = word.replace("'", "")
        word = word.replace("ё", "е")
        word = word.replace("ъ", "ї")
        return word

    def s(self, st, reg, to):
        orig = st
        self.RV = re.sub(reg, to, st)
        return (orig != self.RV)

    def stem_word(self):
        word = self.ukstemmer_search_preprocess(self.word)
        if not re.search('[аеиоуюяіїє]', word):
            stem = word
        else:
            p = re.search(self.rvre, word)
            start = word[0:p.span()[1]]
            self.RV = word[p.span()[1]:]

            # Step 1
            if not self.s(self.RV, self.perfectiveground, ''):

                self.s(self.RV, self.reflexive, '')
                if self.s(self.RV, self.adjective, ''):
                    self.s(self.RV, self.participle, '')
                else:
                    if not self.s(self.RV, self.verb, ''):
                        self.s(self.RV, self.noun, '')
            # Step 2
            self.s(self.RV, 'и$', '')

            # Step 3
            if re.search(self.derivational, self.RV):
                self.s(self.RV, 'ость$', '')

            # Step 4
            if self.s(self.RV, 'ь$', ''):
                self.s(self.RV, 'ейше?$', '')
                self.s(self.RV, 'нн$', u'н')

            stem = start + self.RV
        return stem

In [3]:
df = pd.read_csv('../db.csv', index_col=['id'])
df['parsed_text'] = df.parsed_text.str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2179 entries, 1147 to 5187
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   parsed_text    2179 non-null   object
 1   propaganda     2179 non-null   object
 2   language_code  2179 non-null   object
dtypes: object(3)
memory usage: 68.1+ KB


In [4]:
df.iloc[[80, 266]] = None
df.dropna(inplace=True)

In [5]:
temp = list(map(word_tokenize, df.parsed_text.values))
df['parsed_text'] = temp
df

,parsed_text,propaganda,language_code
id,,,
1147,"[​​китайська, жіночка-кішка, продовжує, намага...",f,ua
2,"[1991, :, россия🇸🇰👍, —, давайте, не, будем, сс...",t,rus
4,"[🚨мысли, вслух, каждого, росисянина🍾🇸🇮, сейчас...",t,rus
5,"[комментарий, пескова, по, поводу, диалога, эс...",t,rus
6,"[реакции, мы, решили, включить, ,, ставим, лай...",t,rus
...,...,...,...
5182,"[#, #, аналiтика, поки, прості, українці, раді...",t,ua
5183,"[#, #, чутки, опісля, того, ,, що, сталося, на...",t,ua
5185,"[⚡️⚡️⚡️, #, #, інсайд, наше, джерело, доповіда...",t,ua


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2177 entries, 1147 to 5187
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   parsed_text    2177 non-null   object
 1   propaganda     2177 non-null   object
 2   language_code  2177 non-null   object
dtypes: object(3)
memory usage: 68.0+ KB


In [7]:
def stemming(parsed_text):
  stem_parsed_text = []
  for word in parsed_text:
    ukrainian_stemmer = UkrainianStemmer(word)
    stem_parsed_text.append(ukrainian_stemmer.stem_word())

  return ' '.join(stem_parsed_text)

In [8]:
print(df.parsed_text)
df['parsed_text'] = df['parsed_text'].apply(lambda x: stemming(x))
print(df.parsed_text)

id
1147    [​​китайська, жіночка-кішка, продовжує, намага...
2       [1991, :, россия🇸🇰👍, —, давайте, не, будем, сс...
4       [🚨мысли, вслух, каждого, росисянина🍾🇸🇮, сейчас...
5       [комментарий, пескова, по, поводу, диалога, эс...
6       [реакции, мы, решили, включить, ,, ставим, лай...
                              ...                        
5182    [#, #, аналiтика, поки, прості, українці, раді...
5183    [#, #, чутки, опісля, того, ,, що, сталося, на...
5185    [⚡️⚡️⚡️, #, #, інсайд, наше, джерело, доповіда...
5186    [#, #, чутки, згідно, наших, джерел, в, оп, ,,...
5187    [#, #, інсайд, наш, експерт, ківа, не, просто,...
Name: parsed_text, Length: 2177, dtype: object
id
1147    ​​китайськ жіночка-кішк продовж намаг в пропаг...
2       1991 : россия🇸🇰👍 — давайт не буд ссорит , мы в...
4       🚨мысли вслух кажд росисянина🍾🇸🇮 сейчас слов ад...
5       комментар песк по повод диалог эстон и зелен :...
6       реакц мы решил включит , став лайк за сильн ро...
                   

In [9]:
my_vectorizer = CountVectorizer()
# print(df['parsed_text'])
# my_vectorizer.fit(df['parsed_text'])

In [10]:
df['parsed_text'] = my_vectorizer.fit_transform(df['parsed_text']).toarray()
x, y = df['parsed_text'].values.reshape(-1, 1), df['propaganda'].apply(lambda y: 0 if y=='f' else 1).values.astype(int)
print(df['propaganda'].apply(lambda y: 0 if y=='f' else 1).describe())
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape, y_train.shape)

count    2177.000000
mean        0.369775
std         0.482855
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: propaganda, dtype: float64
(1523, 1) (1523,)


In [11]:
# df['new_col'] = df['parsed_text'].apply(lambda x: x.shape[1])
# print(df['new_col'].describe())

In [12]:
model = LogisticRegression()
model.fit(x_train, y_train)
x, y

(array([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]), array([0, 1, 1, ..., 1, 1, 1]))

In [13]:
model.score(x_test, y_test)

0.6422018348623854